In [2]:
import numpy as np
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

from datetime import datetime
from dateutil.relativedelta import relativedelta

import pyproj


import descartes
%matplotlib inline
import matplotlib.pyplot as plt

import shapely
import contextily as cx

import open_cp.data
import open_cp.geometry
import open_cp.plot
import matplotlib.collections
import open_cp.kernels

import sepp.sepp_grid
import datetime

from tqdm import tqdm

In [3]:
#read in crime data and clean
crime = pd.read_csv('2018LAcrime.csv')
crime['DATE OCC'] = crime['DATE OCC'].apply(lambda x: datetime.datetime.strptime(x,  '%Y-%m-%d %H:%M:%S'))
#crime = crime[(crime['DATE OCC'].dt.year == 2018)|(crime['DATE OCC'].dt.year == 2017)]
crime = crime[~((crime.LAT==0)&(crime.LON==0))]